In [1]:
# Carga librerías
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Lectura datos iniciales y arreglo nombre variables
reporte_diario_campaña=pd.read_excel("datos_reales_campaña.xlsx").rename(columns={"REAL CALLS":"real_calls","REAL AHT":"real_aht","StateHoliday":"state_holiday"})
reporte_diario_campaña.columns = reporte_diario_campaña.columns.str.lower()
reporte_diario_campaña = reporte_diario_campaña.apply(lambda col: col.map(lambda x: x.lower().replace(' ', '_') if isinstance(x, str) else x))
reporte_diario_campaña["real_aht"]= round(reporte_diario_campaña["real_aht"],2)

In [3]:
# Visualización primeros datos
reporte_diario_campaña.head() 

,fecha,state_holiday,negocio,linea,real_calls,real_aht
0,2020-02-01,0,campaña_1,linea_1,293,338.18
1,2020-02-02,0,campaña_1,linea_1,29,370.12
2,2020-02-03,0,campaña_1,linea_1,2881,395.45
3,2020-02-04,0,campaña_1,linea_1,2776,404.66
4,2020-02-05,0,campaña_1,linea_1,2586,399.82


In [4]:
# Resumen estadístico variables numéricas
reporte_diario_campaña.describe()

,fecha,state_holiday,real_calls,real_aht
count,6330,6330.000000,6330.000000,6330.000000
mean,2022-06-03 05:22:48.341232128,0.034913,1101.203476,329.518014
min,2020-02-01 00:00:00,0.000000,0.000000,0.000000
25%,2021-07-12 00:00:00,0.000000,113.000000,285.532500
50%,2022-07-14 00:00:00,0.000000,408.500000,339.730000
75%,2023-05-26 18:00:00,0.000000,2125.500000,387.327500
max,2024-04-07 00:00:00,2.000000,5652.000000,803.530000
std,NaN,0.217457,1270.833359,88.316354


In [5]:
# Información sobre el df (tipos de datos)
reporte_diario_campaña.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6330 entries, 0 to 6329
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   fecha          6330 non-null   datetime64[ns]
 1   state_holiday  6330 non-null   int64         
 2   negocio        6330 non-null   object        
 3   linea          6330 non-null   object        
 4   real_calls     6330 non-null   int64         
 5   real_aht       6330 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 296.8+ KB


In [6]:
# Variables temporales adicionales 
reporte_diario_campaña["dia"]=reporte_diario_campaña["fecha"].dt.day
reporte_diario_campaña["año"]=reporte_diario_campaña["fecha"].dt.year
reporte_diario_campaña["dia_semana"]=reporte_diario_campaña["fecha"].dt.weekday
reporte_diario_campaña["mes"]=reporte_diario_campaña["fecha"].dt.month

In [7]:
# Agregación festivos como día de la semana 7 
reporte_diario_campaña.loc[reporte_diario_campaña['state_holiday'] == 1, 'dia_semana'] = 7

In [8]:
# Asignación índices
reporte_diario_campaña['time_idx'] = reporte_diario_campaña['fecha'].rank(method='dense').astype(int) - 1

In [9]:
# Verificación datos nulos
reporte_diario_campaña.isnull().sum()

fecha            0
state_holiday    0
negocio          0
linea            0
real_calls       0
real_aht         0
dia              0
año              0
dia_semana       0
mes              0
time_idx         0
dtype: int64

In [10]:
# Condicones negocio 
reporte_diario_campaña.loc[(reporte_diario_campaña["linea"] == "linea_4") & (reporte_diario_campaña["fecha"]<"2023-03-11"), 'real_calls'] = np.nan
reporte_diario_campaña.loc[(reporte_diario_campaña["linea"] == "linea_5") & (reporte_diario_campaña["fecha"]<"2021-01-01"), 'real_calls'] = np.nan

reporte_diario_campaña.loc[(reporte_diario_campaña["linea"] == "linea_3") & (reporte_diario_campaña["fecha"]<"2022-10-01"), 'real_aht'] = np.nan
reporte_diario_campaña.loc[(reporte_diario_campaña["linea"] == "linea_4") & (reporte_diario_campaña["fecha"]<"2022-06-13"), 'real_aht'] = np.nan

In [11]:
lineas = reporte_diario_campaña['linea'].unique()

# Crear un Dropdown widget
dropdown = widgets.Dropdown(
    options=lineas,
    value=lineas[0],
    description='Línea:',
    disabled=False,
)

# Función para actualizar la gráfica
def update_plot(linea):
    df = reporte_diario_campaña[reporte_diario_campaña["linea"] == linea]
    plt.figure(figsize=(14, 7))
    
    plt.subplot(2, 1, 1)
    plt.plot(df['fecha'], df['real_calls'], label='Llamadas', color='blue')
    plt.title('Llamadas')
    plt.xlabel('Fecha')
    plt.ylabel('Llamadas')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Conectar el widget con la función de actualización
out = widgets.interactive_output(update_plot, {'linea': dropdown})

# Mostrar el widget y la salida gráfica
display(dropdown, out)

Dropdown(description='Línea:', options=('linea_1', 'linea_2', 'linea_3', 'linea_4', 'linea_5'), value='linea_1…

Output()

In [12]:
lineas = reporte_diario_campaña['linea'].unique()

# Crear un Dropdown widget 
dropdown = widgets.Dropdown(
    options=lineas,
    value=lineas[0],
    description='Línea:',
    disabled=False,
)

# Función para actualizar la gráfica
def update_plot(linea):
    df = reporte_diario_campaña[reporte_diario_campaña["linea"] == linea]
    plt.figure(figsize=(14, 7))
    
    plt.subplot(2, 1, 1)
    plt.plot(df['fecha'], df['real_aht'], label='AHT', color='blue')
    plt.title('AHT')
    plt.xlabel('Fecha')
    plt.ylabel('AHT')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Conectar el widget con la función de actualización
out = widgets.interactive_output(update_plot, {'linea': dropdown})

# Mostrar el widget y la salida gráfica
display(dropdown, out)

Dropdown(description='Línea:', options=('linea_1', 'linea_2', 'linea_3', 'linea_4', 'linea_5'), value='linea_1…

Output()

In [13]:
# Cantidad de datos de cada linea
reporte_diario_campaña.groupby('linea').agg(
    Cantidad_Real_AHT=('real_aht', lambda x: x.dropna().count()),
    Cantidad_Real_Calls=('real_calls', lambda x: x.dropna().count()),
    Fecha_Inicial=('fecha', 'min'),
    Fecha_Final=('fecha', 'max')
).reset_index()


,linea,Cantidad_Real_AHT,Cantidad_Real_Calls,Fecha_Inicial,Fecha_Final
0,linea_1,1528,1528,2020-02-01,2024-04-07
1,linea_2,1528,1528,2020-02-01,2024-04-07
2,linea_3,555,828,2022-01-01,2024-04-07
3,linea_4,665,394,2021-10-01,2024-04-07
4,linea_5,1526,1191,2020-02-01,2024-04-05


In [14]:
# Definición función de limpieza e interpolación de outliers
def limpieza(data, target):
    new = pd.DataFrame()
    data = data.dropna(subset=[target])
    reporte_diario_linea = data.groupby(['linea', 'año', 'dia_semana'])

    for (linea, año, dia_semana), reporte_linea in reporte_diario_linea:
        # Encuentra los outliers
        model = IsolationForest(random_state=0, contamination=0.12)
        reporte_linea["out_"+ target] = model.fit_predict(reporte_linea[[target]]) == -1

        # Cambia a NaN los outliers
        reporte_linea["interpolado_" + target] = reporte_linea[target]
        reporte_linea.loc[reporte_linea["out_"+ target], "interpolado_" + target] = np.nan

        # Los valores que son NaN los cambia por el promedio del dato siguiente con el anterior
        reporte_linea["interpolado_" + target] = reporte_linea["interpolado_" + target].fillna(
            (reporte_linea["interpolado_" + target].shift(-1) + reporte_linea["interpolado_" + target].shift(1)) / 2
        )

        # Cambia los que siguen siendo NaN por los vecinos más cercanos
        imputer = KNNImputer(n_neighbors=3)
        reporte_linea["interpolado_" + target] = imputer.fit_transform(reporte_linea[["interpolado_" + target]])

        # Ajustar tipo de dato target
        if target == "real_calls":
            reporte_linea["interpolado_" + target] = round(reporte_linea["interpolado_" + target]).astype(int)
        else:
            reporte_linea["interpolado_" + target] = round(reporte_linea["interpolado_" + target],2)

        # Concatena a la base final
        new = pd.concat([new, reporte_linea], ignore_index=True, sort=False)

    # Ordena el DataFrame final
    new = new.sort_values(["linea","fecha"])
    return new


In [15]:
# Aplicación funciónn creada 
reporte_diario_campaña_limpio=pd.merge(
    limpieza(reporte_diario_campaña,"real_aht"),
    limpieza(reporte_diario_campaña,"real_calls"),
    how='outer').sort_values(["linea","fecha"])


In [16]:
# Visualización nuevo df
reporte_diario_campaña_limpio

,fecha,state_holiday,negocio,linea,real_calls,real_aht,dia,año,dia_semana,mes,time_idx,out_real_aht,interpolado_real_aht,out_real_calls,interpolado_real_calls
0,2020-02-01,0,campaña_1,linea_1,293.0,338.18,1,2020,5,2,0,False,338.18,False,293.0
1,2020-02-02,0,campaña_1,linea_1,29.0,370.12,2,2020,6,2,1,False,370.12,False,29.0
2,2020-02-03,0,campaña_1,linea_1,2881.0,395.45,3,2020,0,2,2,False,395.45,False,2881.0
3,2020-02-04,0,campaña_1,linea_1,2776.0,404.66,4,2020,1,2,3,False,404.66,False,2776.0
4,2020-02-05,0,campaña_1,linea_1,2586.0,399.82,5,2020,2,2,4,False,399.82,False,2586.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797,2024-04-01,0,campaña_1,linea_5,81.0,116.55,1,2024,0,4,1521,True,334.34,True,978.0
5798,2024-04-02,0,campaña_1,linea_5,82.0,128.80,2,2024,1,4,1522,True,335.65,True,878.0
5799,2024-04-03,0,campaña_1,linea_5,2.0,0.00,3,2024,2,4,1523,True,331.04,True,812.0
5800,2024-04-04,0,campaña_1,linea_5,2.0,0.00,4,2024,3,4,1524,True,332.98,True,759.0


In [17]:
lineas = reporte_diario_campaña_limpio['linea'].unique()

# Crear un Dropdown widget
dropdown = widgets.Dropdown(
    options=lineas,
    value=lineas[0],
    description='Línea:',
    disabled=False,
)

# Función para actualizar la gráfica
def update_plot(linea):
    df = reporte_diario_campaña_limpio[reporte_diario_campaña_limpio["linea"] == linea].sort_values(["fecha"])
    plt.figure(figsize=(14, 7))
    
    plt.subplot(2, 1, 1)
    plt.plot(df['fecha'], df['real_calls'], label='Real', color='blue')
    plt.plot(df['fecha'], df['interpolado_real_calls'], label='Interpolado', color='red', linestyle='--')
    plt.title('Real Vs Interpolado')
    plt.xlabel('Fecha')
    plt.ylabel('Llamadas')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Conectar el widget con la función de actualización
out = widgets.interactive_output(update_plot, {'linea': dropdown})

# Mostrar el widget y la salida gráfica
display(dropdown, out)

Dropdown(description='Línea:', options=('linea_1', 'linea_2', 'linea_3', 'linea_4', 'linea_5'), value='linea_1…

Output()

In [18]:
lineas = reporte_diario_campaña_limpio['linea'].unique()

# Crear un Dropdown widget 
dropdown = widgets.Dropdown(
    options=lineas,
    value=lineas[0],
    description='Línea:',
    disabled=False,
)

# Función para actualizar la gráfica
def update_plot(linea):
    df = reporte_diario_campaña_limpio[reporte_diario_campaña_limpio["linea"] == linea]
    plt.figure(figsize=(14, 7))
    
    plt.subplot(2, 1, 1)
    plt.plot(df['fecha'], df['real_aht'], label='Real', color='blue')
    plt.plot(df['fecha'], df['interpolado_real_aht'], label='Interpolado', color='red', linestyle='--')

    plt.title('Real VS Interpolado')
    plt.xlabel('Fecha')
    plt.ylabel('AHT')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Conectar el widget con la función de actualización
out = widgets.interactive_output(update_plot, {'linea': dropdown})

# Mostrar el widget y la salida gráfica
display(dropdown, out)

Dropdown(description='Línea:', options=('linea_1', 'linea_2', 'linea_3', 'linea_4', 'linea_5'), value='linea_1…

Output()

In [19]:
reporte_diario_campaña_limpio.to_excel("reporte_diario_campaña_limpio.xlsx")